<a href="https://colab.research.google.com/github/Yiqing-Wang-05/design_graph/blob/main/Redesign_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving GlobalLandTemperaturesByCountry.csv to GlobalLandTemperaturesByCountry.csv


In [8]:
# Step 0: Install necessary packages (only needed once in Colab)
# !pip install plotly pandas openpyxl

# Step 1: Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px

# Step 2: Load the dataset (make sure you’ve uploaded the CSV to Colab first)
df = pd.read_csv("GlobalLandTemperaturesByCountry.csv")

# Step 3: Clean the dataset by removing missing temperature values and extracting year
df['Year'] = pd.to_datetime(df['dt']).dt.year
df_clean = df.dropna(subset=['AverageTemperature'])

# Step 4: Group by country and year, calculate average temperature
df_grouped = df_clean.groupby(['Country', 'Year'])['AverageTemperature'].mean().reset_index()

# Step 5: Limit to post-1900 data for meaningful animation
df_grouped = df_grouped[df_grouped['Year'] >= 1900]

# Step 6: Add narrative cues to certain country-year combinations
df_grouped["Narrative"] = np.where(
    (df_grouped["Country"] == "France") & (df_grouped["Year"] == 2003),
    "2003 European Heatwave",
    np.where(
        (df_grouped["Country"] == "Russia") & (df_grouped["Year"] == 2010),
        "2010 Russian Heatwave",
        ""
    )
)

# Step 7: Define a custom color scale to reflect emotional urgency
custom_colorscale = [
    [0.0, "#f2f2f2"],   # Bone white (calm)
    [0.5, "#ff7f00"],   # Orange red (warning)
    [1.0, "#800080"]    # Flaming purple (crisis)
]

# Step 8: Create the interactive animated map
fig = px.choropleth(
    df_grouped,
    locations="Country",
    locationmode="country names",
    color="AverageTemperature",
    hover_name="Country",
    hover_data={
        "Year": True,
        "AverageTemperature": ':.2f',
        "Narrative": True
    },
    animation_frame="Year",
    color_continuous_scale=custom_colorscale,
    range_color=(df_grouped['AverageTemperature'].min(), df_grouped['AverageTemperature'].max()),
    title="Feeling the Heat: Global Land Temperature by Country (1900–2013)"
)

# Step 9: Style the map and show
fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True),
    coloraxis_colorbar=dict(title="Avg Temp (°C)"),
    width=1100,
    height=600
)

fig.show()